### This example shows how to use read/write functions for <ul>
<li>Input </li>
<li>Grid </li>
<li>Mask </li>
<li>Dep </li>
<li>Forcing </li>
</ul>

In [ ]:
#for first run install package
#pip install -e ..

In [ ]:
import os
import sys
from hydromt_sfincs.sfincs import Sfincs

model = "sfincs_coastal"

### Input

In [ ]:
#read
sf = Sfincs()
sf.read_input_file(os.path.join(model, "sfincs.inp"))
inp = sf.inp

In [ ]:
# eventually change input and write again
# sf.inp.dx= 10
# sf.write_input_file(os.path.join(model,"sfincs_v2.inp"))

### Grid

In [ ]:
#grid is automatically read in from sfincs.inp
grid = sf.grid

Compare different defenitions of grid

In [ ]:
# #Compare with previous definition of grid
# import math
# import numpy as np

# #old defenitions
# cosrot = math.cos(inp.rotation * math.pi / 180)
# sinrot = math.sin(inp.rotation * math.pi / 180)

# xx = np.linspace(
#     0.5 * inp.dx, inp.mmax * inp.dx - 0.5 * inp.dx, num=inp.mmax
# )
# yy = np.linspace(
#     0.5 * inp.dy, inp.nmax * inp.dy - 0.5 * inp.dy, num=inp.nmax
# )

# xg0, yg0 = np.meshgrid(xx, yy)
# xg = inp.x0 + xg0 * cosrot - yg0 * sinrot
# yg = inp.y0 + xg0 * sinrot + yg0 * cosrot

# #new defenitions
# x = grid.coordinates["x"]
# y = grid.coordinates["y"]

# xn,yn = np.meshgrid(x,y)

# #differences
# print(sum(sum(xg-xn)))
# print(sum(sum(yg-yn)))

### Mask

In [ ]:
#read mask
sf.read_mask()
sf.grid.mask.plot.imshow()

In [ ]:
#write
sf.write_mask("sfincs_v2.msk", "sfincs_v2.ind")

### Dep and Manning

In [ ]:
dep = sf.read_map("dep")
sf.grid.data["dep"].raster.mask_nodata().plot.imshow()

In [ ]:
#change depth and write away
sf.grid.data["dep"] = sf.grid.data["dep"] + 10
sf.write_map("dep", "sfincs_v2.dep")

In [ ]:
#read again and plot
sf.read_map("dep")
sf.grid.data["dep"].raster.mask_nodata().plot.imshow()

In [ ]:
man = sf.read_map("manning")
sf.grid.data["manning"].raster.mask_nodata().plot.imshow()

### Forcing

In [ ]:
sf.read_forcing_1d('waterlevel')

In [ ]:
#boundary points
sf.forcing['bnd']

In [ ]:
#boundary timeseries
sf.forcing['bzs']

In [ ]:
#add 0.5m to all boundary water levels and write again
sf.forcing['bzs'] = sf.forcing['bzs'] + 0.5
sf.write_forcing_1d('waterlevel', "sfincs_v2.bzs", "sfincs_v2.bnd")

In [ ]:
sf.write_input_file("sfincs_v2.inp")